# **Chinook Record Store: Answering Business Questions using SQL**

*Rishabh Sharma*

*Updated - 01/31/2022*

In this project, we try to answer some business questions related to the *Chinook Record Store,* which sells albums associated with different artists and genres, and have customers from all around the world. 

**Lets first connect our jupyter notebook to the dataset `chinook.db` we need to work with.**

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

This confirms that our connection with the database is successful. We can now get familiar with our dataset by looking how many tables it consist, what are the types of those tables (table or view)?

In [3]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ('table', 'view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Looks like we have about 12 tables in the `chinook.db` dataset. We can also preview few rows from random tables to familiarize with what actually is the content of these tables.


In [4]:
%%sql

SELECT * 
  FROM customer
 LIMIT 3;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


It also have a table named invoice, lets preview it.

In [5]:
%%sql
SELECT *
  FROM invoice
 LIMIT 3;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


Now we know the structure of some tables we can start answering some business questions for our store *Chinook.*

**1. Find out which genres sell the most tracks in the USA and recommend top three artists whose album should store buy from the following options available :**


1. Regal - *Hip-Hop*
2. Red Tone	- *Punk*
3. Meteor and the Girls - *Pop*
4. Slim Jim Bites - *Blues*

In [6]:
%%sql
 
WITH songs_sold_usa AS
 
    (SELECT il.* 
       FROM invoice_line AS il
      INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
      INNER JOIN customer AS c ON c.customer_id = i.customer_id
      WHERE c.country = 'USA'
    )
    
SELECT g.name,
       COUNT(ssu.invoice_line_id) AS tracks_sold,
       COUNT(ssu.invoice_line_id) * 100.0 / ( SELECT COUNT(*) FROM songs_sold_usa) AS Percentage_contribution
  FROM songs_sold_usa AS ssu
 INNER JOIN track AS t ON t.track_id = ssu.track_id
 INNER JOIN genre As g ON g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC;


Done.


name,tracks_sold,Percentage_contribution
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Based on the store's history of sold tracks and among the available options, store should buy the albums of following artists:

1. Red Tone - *Punk*
2. Slim Jim Bites - *Blues*
3. Meteor and the Girls - *Pop*

*Note - The Rock Genre is the most profitable among all and accounts for about 53.3% of the total sales. Store should find ways to buy tracks from artists who produces tracks under the 'Rock' category.*

**2. Find out the dollar amount of sales secured by each sales support agent working at Chinook store to determine their relative performance to each other and find any attributes if relevant to the analysis.**

In [7]:
%%sql

WITH valuable_customer AS
        (SELECT (c.first_name || ' ' || c.last_name) AS name,
                c.customer_id,
                c.support_rep_id,
                SUM(i.total) AS total
           FROM customer AS c
          INNER JOIN invoice AS i ON i.customer_id = c.customer_id
          GROUP BY 1
          ORDER BY 4 DESC
        )

SELECT e.employee_id,
       (e.first_name || ' ' || e.last_name) AS sales_rep,
       SUM(vc.total) AS total_sales,
       e.hire_date
  FROM employee AS e
 INNER  JOIN valuable_customer AS vc ON e.employee_id = vc.support_rep_id
 GROUP BY 2
 ORDER BY 3 DESC;

Done.


employee_id,sales_rep,total_sales,hire_date
3,Jane Peacock,1731.5099999999998,2017-04-01 00:00:00
4,Margaret Park,1584.0,2017-05-03 00:00:00
5,Steve Johnson,1393.92,2017-10-17 00:00:00


Above table implies that the top sales rep with highest sales amount is *Jane Peacock.* The `hire_date` field also illustrates that the difference between the top employee (Jane) and bottom employee (Steve) can be attributed to the working duration of the emplyee at the store. This states that longer the employee is working at the Chinook store, the better he/she gets for securing the sale deals.

**3. Chinook store wants to find out the most profitable countries based on their sales data. Collate data from different countries including specific fields / For each country include:**

1. Total Number of customers
2. Total Value of sales
3. Average Value of sales per customer
4. Average Order Value


In [22]:
%%sql

WITH number_customers AS     
    (SELECT COUNT(DISTINCT(c.customer_id)) AS number,
            c.country
       FROM customer AS c
      GROUP BY 2
      ORDER BY 1 DESC
     ),
    
value AS

     (SELECT SUM(i.total) AS total,
             c.country,
             COUNT(DISTINCT(i.invoice_id)) AS number_orders
        FROM customer AS c
       INNER JOIN invoice AS i ON i.customer_id = c.customer_id
       GROUP BY 2
       ORDER BY 1
      )
    
SELECT n.country AS 'Country',
       n.number AS 'No. of customers',
       v.total AS 'Total Value of sales',
       (v.total / n.number) AS 'Average Value of sales / customer',
       (v.total / v.number_orders) AS 'Average order value'
  FROM number_customers AS n
  JOIN value AS v ON n.country = v.country
 GROUP BY 1
 ORDER BY 3 DESC;

Done.


Country,No. of customers,Total Value of sales,Average Value of sales / customer,Average order value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Ireland,1,114.83999999999997,114.83999999999997,8.833846153846151


Above tabel shows depicts that **USA** is the most profitable country for the store with an average order value of *7.94* followed by **Canada** with average order value of *7.05.* 

**5. The manager at Chinook record store is confused with the different formats (media type) made available by the vendor. Find the appropriate media type ratio to buy for future albums.**

In [42]:
%%sql
 
WITH temp_table AS
 
    (SELECT il.* 
       FROM invoice_line AS il
      INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
      INNER JOIN customer AS c ON c.customer_id = i.customer_id 
    )
    
SELECT m.name,
       COUNT(tt.invoice_line_id) AS tracks_sold,
       COUNT(tt.invoice_line_id) * 100.0 / ( SELECT COUNT(*) FROM temp_table) AS Percentage_contribution
  FROM temp_table AS tt
 INNER JOIN track AS t ON t.track_id = tt.track_id
 INNER JOIN media_type As m ON m.media_type_id = t.media_type_id
 GROUP BY 1
 ORDER BY 2 DESC;


Done.


name,tracks_sold,Percentage_contribution
MPEG audio file,4259,89.53121715366828
Protected AAC audio file,439,9.228505360521337
Purchased AAC audio file,35,0.7357578305654825
AAC audio file,21,0.44145469833928946
Protected MPEG-4 video file,3,0.06306495690561278


The above table suggests that the manager should buy the *MPEG audio file* format the most (accounting for about 89% of the total sales) followed by the *Protected AAC audio file* (accounting for about 9% of all media types (formats) available.